# Deep Learning Models

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import shap
import lime
import lime.lime_image
from keras.models import load_model
from skimage.segmentation import mark_boundaries
import cv2

# Load all models
models = {
    "Deeper CNN": load_model("cnn_deeper.h5"),
    "RNN": load_model("rnn.h5"),
    "LSTM": load_model("lstm.h5"),
    "Capsule Network": load_model("capsule.h5"),
}

# Load test image
def load_test_image(path="test_digit.png"):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype("float32") / 255.0
    return img

# Preprocess for model input
def preprocess_for_model(img):
    return np.expand_dims(np.expand_dims(img, axis=-1), axis=0)


## Grad-CAM

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    grad_model.summary()

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        output = predictions[:, pred_index]

    grads = tape.gradient(output, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (28, 28))
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    img_color = cv2.cvtColor(np.uint8(img * 255), cv2.COLOR_GRAY2BGR)
    overlay = cv2.addWeighted(img_color, 1 - alpha, heatmap_colored, alpha, 0)
    plt.imshow(overlay)
    plt.title("Grad-CAM Overlay")
    plt.axis('off')
    plt.show()

## LIME

In [ ]:
def explain_with_lime(image, model):
    explainer = lime.lime_image.LimeImageExplainer()
    def predict_fn(imgs):
        imgs = np.array([
            cv2.cvtColor(cv2.resize(i, (28, 28)), cv2.COLOR_RGB2GRAY)
            for i in imgs
        ])
        imgs = imgs[..., np.newaxis] / 255.0
        return model.predict(imgs)

    explanation = explainer.explain_instance(
        image=cv2.cvtColor((image * 255).astype("uint8"), cv2.COLOR_GRAY2RGB),
        classifier_fn=predict_fn,
        top_labels=1,
        hide_color=0,
        num_samples=500
    )
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],
        positive_only=True,
        num_features=5,
        hide_rest=False
    )
    plt.imshow(mark_boundaries(temp / 255.0, mask))
    plt.title("LIME Explanation")
    plt.axis('off')
    plt.show()


## SHAP

In [ ]:
def explain_with_shap(model, image):
    background = np.stack([image for _ in range(100)])
    explainer = shap.GradientExplainer(model, background)
    shap_values = explainer.shap_values(np.expand_dims(image, axis=0))
    shap.image_plot(shap_values, np.expand_dims(image, axis=0))

## Integrated Gradients

In [ ]:
def integrated_gradients(model, input_image, baseline=None, steps=50):
    input_image = tf.convert_to_tensor(input_image[0], dtype=tf.float32)

    if baseline is None:
        baseline = tf.zeros_like(input_image)
    else:
        baseline = tf.convert_to_tensor(baseline[0], dtype=tf.float32)

    interpolated = tf.stack([
        baseline + (float(i) / steps) * (input_image - baseline)
        for i in range(steps + 1)
    ])

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        predictions = model(interpolated)
        pred_index = tf.argmax(predictions[-1])
        outputs = predictions[:, pred_index]

    grads = tape.gradient(outputs, interpolated)
    avg_grads = tf.reduce_mean(grads, axis=0)

    integrated_grads = (input_image - baseline) * avg_grads
    return integrated_grads.numpy()

## Saliency Map

In [ ]:
def saliency_map(model, input_image):
    input_image = tf.convert_to_tensor(input_image, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(input_image)
        predictions = model(input_image)
        pred_index = tf.argmax(predictions[0])
        output = predictions[:, pred_index]

    grads = tape.gradient(output, input_image)
    saliency = tf.abs(grads)
    return saliency.numpy()[0, ..., 0]

### Main

In [ ]:
# Load test image
img_raw = load_test_image()
img_input = preprocess_for_model(img_raw)

# Define the last conv layer name per CNN model
last_conv_layers = {
    "Deeper CNN": "conv2d_2",      # <- check via model.summary() if different
    "Capsule Network": "conv2d_7",   # <- also confirm via model.summary()
}

# Loop through models and apply XAI
for model_name, model in models.items():
    print(f"\n🔍 Explaining predictions for: {model_name}")
    
    # LIME
    try:
        print("➡ LIME")
        explain_with_lime((img_raw * 255).astype("uint8"), model)
    except Exception as e:
        print("LIME failed:", e)

    # Grad-CAM (only for CNNs)
    if model_name in last_conv_layers:
        try:
            print("➡ Grad-CAM")
            heatmap = make_gradcam_heatmap(img_input, model, last_conv_layers[model_name])
            display_gradcam(img_raw, heatmap)
        except Exception as e:
            print("Grad-CAM failed:", e)

In [ ]:
def display_explanation_map(img, explanation, title):
    plt.imshow(img, cmap='gray')
    plt.imshow(explanation, cmap='inferno', alpha=0.6)
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# Preprocessed input (1, 28, 28, 1)
input_img = preprocess_for_model(img_raw)

# For Deeper CNN
ig = integrated_gradients(models["Deeper CNN"], input_img)
sal = saliency_map(models["Deeper CNN"], input_img)

display_explanation_map(img_raw, np.squeeze(ig), "Integrated Gradients")
display_explanation_map(img_raw, sal, "Saliency Map")